## Risk Analysis & Modelling
---
# New York Taxi & Shootings (Contd.)
#### *Quantitative Geospatial Analysis* 

In here, we aim to analyse the risk of our current situation with a statistical model I built, and also analyse the risk of a proposed solution to reduce the risk of drivers and passengers in a given time of day. The Risk Model I built is,

$$ \mathbf{risk}_{t} = \mathbf{Shooting}_{t} \cdot \mathbf{Taxi}_{t} = \sum_{b,\ time = t} Shooting_b \times Taxi_b $$

## 1. Import Libraries

In [27]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, col

import pandas as pd
import geopandas as gpd

import seaborn as sns
import matplotlib.pyplot as plt

import folium 
from folium.plugins import HeatMap, HeatMapWithTime # Heatmap (frame) and HeatMapWithTime (animation)

import warnings
warnings.filterwarnings("ignore")

# Start the spark context
sc = SparkContext.getOrCreate(conf=swan_spark_conf)

# create a spark session (which will run spark jobs)
spark = SparkSession.builder.getOrCreate()

spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

## 2. Import Datasets

In [ ]:
# Import FHV Data and only retaining the attributes mentioned in the report.
taxi = spark.read.csv("../raw_data/fhvhv_tripdata_2019-04.csv", header = True) \
                .unionByName(spark.read.csv("../raw_data/fhvhv_tripdata_2019-05.csv", header = True))\
                .unionByName(spark.read.csv("../raw_data/fhvhv_tripdata_2019-06.csv", header = True))\
                .drop("hvfhs_license_num", "dispatching_base_num", "SR_Flag")\
                .select(col("pickup_datetime").cast("Timestamp"),col("dropoff_datetime").cast("Timestamp"),\
                        col("PULocationID").cast("Integer"),col("DOLocationID").cast("Integer"))
taxi = taxi.withColumn('pickup_datetime', date_format('pickup_datetime', 'HH:mm:ss')).withColumn('dropoff_datetime', date_format('pickup_datetime', 'HH:mm:ss'))

# Import Taxi Zone Data Dictionary and only retaining the attributes mentioned in the report.
taxi_zones = gpd.read_file("../raw_data/taxi_zones")

# Merge Taxi and Taxi Zone Data Dictionary to make pick up and drop off 
pick_up  = gpd.GeoDataFrame(pd.merge(taxi.groupby('PULocationID').count().toPandas(), taxi_zones, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1) 
drop_off = gpd.GeoDataFrame(pd.merge(taxi.groupby('DOLocationID').count().toPandas(), taxi_zones, left_on='DOLocationID', right_on='LocationID')).drop('DOLocationID',axis=1) 

# Import Shooting Data and only retaining the attributes and rows mentioned in the report.
shooting  = spark.read.csv("../raw_data/NYPD_Shooting_Incident_Data__Historic_.csv", header = True)\
                .select("OCCUR_DATE", "OCCUR_TIME", "BORO", "Latitude", "Longitude")
shooting  = shooting.filter(shooting.OCCUR_DATE.rlike(r'[0456]{2}/\d{2}/2019')).drop('OCCUR_DATE')

## 3. Analyse Data
The Risk Model that I have constructed is a **linear combination** of *the number of cars* and *the number of shootings* in borough in a given time. 
This model is sound as the impact of any one of the shooters puts all of the cars, which are **independent of eachother**, at **RISK**. Hence a linear combination of shooter and cars is appropirate. Where one shooter is one 
unit of risk, and we scale that unit of risk by the number of cars. The model mathematically is,

$$ \mathbf{risk}_{t} = \mathbf{Shooting}_{t} \cdot \mathbf{Taxi}_{t} = \sum_{b,\ time = t} Shooting_b \times Taxi_b $$

The time is held constant in the summation and we accumlate the counts over all boroughs. This time is the time block indicated below

### 3.1 Partition Shooting and Driving (Taxi) into *Custom Time Block*

In [28]:
shootings = { 
                "shoot_0_4"  : (shooting.where(shooting.OCCUR_TIME.rlike(r'0[0-4]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "shoot_4_9"   : (shooting.where(shooting.OCCUR_TIME.rlike(r'0[4-9]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "shoot_10_13" : (shooting.where(shooting.OCCUR_TIME.rlike(r'1[0-3]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "shoot_13_17" : (shooting.where(shooting.OCCUR_TIME.rlike(r'1[3-7]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "shoot_17_19" : (shooting.where(shooting.OCCUR_TIME.rlike(r'1[7-9]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "shoot_20_24" : (shooting.where(shooting.OCCUR_TIME.rlike(r'2[0-4]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10))
              }

driving = { 
                "taxi_0_4"  : (taxi.where(taxi.pickup_datetime.rlike(r'0[0-4]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "taxi_4_9"   : (taxi.where(taxi.pickup_datetime.rlike(r'0[4-9]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "taxi_10_13" : (taxi.where(taxi.pickup_datetime.rlike(r'1[0-3]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "taxi_13_17" : (taxi.where(taxi.pickup_datetime.rlike(r'1[3-7]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "taxi_17_19" : (taxi.where(taxi.pickup_datetime.rlike(r'1[7-9]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)),\
               "taxi_20_24" : (taxi.where(taxi.pickup_datetime.rlike(r'2[0-4]:\d{2}:\d{2}')), folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10))
              }

### 3.2 Process Data: *GroupBy Shooting and Taxi Counts*
Will take `~3 min` to run due to **BIG Data**, but much better than other data processors as `pyspark` uses multicore processing.

In [31]:
shootings
S = {}
for time in shootings.keys():
    S[time] = shootings[time][0].groupby('BORO')\
                                .count()\
                                .toPandas()
F = {}
for time in driving.keys():
    F[time] = pd.DataFrame.merge(driving[time][0].toPandas(), taxi_zones, left_on="PULocationID", right_on='LocationID')\
                .groupby('borough')\
                .count()['PULocationID']

## Current Situation - Risk Analysis 

 `mat` has the first value as the drivers in a borough along with its number of shooter for each borough for each time. Each row is a *time block* and each column is a *borogh*. Within each borough, the first element is the number of cars and the second is the number of shooters, which is our unit of risk

In [32]:
risk_values = []
mat = []
for t in S.keys():
    s = {"Queens": 0, "Brooklyn": 0, "Bronx": 0, "Manhattan": 0, "Staten Island": 0}
    f = {"Queens": 0, "Brooklyn": 0, "Bronx": 0, "Manhattan": 0, "Staten Island": 0}
    sk = t
    fk = 'taxi' + t[5:]
    risk = 0 
    row = []
    for b in f.keys():
        f[b] += F[fk][b]
        try:
            s[b] += S[sk].loc[S[sk]['BORO'] == b.upper()]['count'].iloc[0]
        except IndexError:
            s[b] += 0
        row.append([f[b], s[b]])
    risk = sum([s[b]*f[b] for b in s.keys()])
    risk_values.append(risk)
    mat.append(row)
    
mat

[[[1357129, 8], [2121824, 20], [804285, 28], [3110077, 18], [67580, 2]],
 [[2528790, 7], [3302149, 11], [1607312, 3], [4791122, 9], [154913, 0]],
 [[2095350, 2], [2863736, 9], [1248414, 3], [4522556, 1], [133424, 0]],
 [[2944520, 8], [4103525, 19], [1886157, 7], [6709523, 5], [197345, 1]],
 [[1936995, 6], [2940786, 24], [1221852, 8], [5250965, 10], [115893, 2]],
 [[2503353, 8], [3904069, 31], [1539039, 25], [6546757, 4], [125579, 2]]]

In [39]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=risk_values, name="Orignal FHV Risk",
                    text=["tweak line smoothness<br>with 'smoothing' in line object"],
                    hoverinfo='text+name',
                    line_shape='spline'))

fig.update_layout(title="Current Shooting Incident Risk Drivers Face",
                  xaxis = dict(
                  tickmode = 'array',
                  tickvals = list(range(9)),
                  ticktext = ["0:00", "4:00", "9:00", "15:00", "17:00", "24:00"])
)

**Take-Aways:** From the *Plot*, it can be seen that the risk dips starts moderately high, dipping at 9am and moves up till 3pm then a slight dip at 5pm, later to only move up till midnight.

## Proposed Plan - Risk Analysis
Now what if we took half of the cars where there are the most risk, and move them to where there is the least risk. Yes this would be an undesirable option for passengers as they wouldn't prefer to be pickup from other locations.
However, they do reduce the risk in some areas. None-the-less, this choice must be analysed first.

In [29]:
mod_mat = mat.copy()

r = 0
for row in mat:
    max_ = 0
    max_i = 0
    min_ = np.inf
    min_i = 0
    i = 0
    for b in row:
        if b[0] * b[1] > max_:
            max_ = b[0] * b[1]
            max_i = i
        if b[0] * b[1] < min_:
            min_ = b[0] * b[1]
            min_i = i
        i += 1
    mod_mat[r][max_i][0] /= 2
    mod_mat[r][min_i][0] += mod_mat[r][max_i][0]
    mod_mat[r] = sum(list(map(lambda x : x[0] * x[1], mod_mat[r])))
    r += 1
    
mod_mat

[75512698.0, 51465310.0, 15680201.0, 79654560.5, 75190257.0, 103354674.75]

In [41]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=risk_values, name="Orignal FHV Risk",
                    text=["tweak line smoothness<br>with 'smoothing' in line object"],
                    hoverinfo='text+name',
                    line_shape='spline'),
             )

fig.add_trace(go.Scatter(y=mod_mat, name="Theoretical Plan FHV Risk ",
                    text=["tweak line smoothness<br>with 'smoothing' in line object"],
                    hoverinfo='text+name',
                    line_shape='spline'),
             )

risk_values = np.array(risk_values)
mod_mat = np.array(mod_mat)

fig.add_trace(go.Scatter(y=risk_values-mod_mat, name="Risk Reduction",
                    text=["tweak line smoothness<br>with 'smoothing' in line object"],
                    hoverinfo='text+name',
                    line_shape='spline',
                    line_dash = 'dash'
             ))
fig.update_layout(
    title="Original vs Proposed Plan Risk (April - June 2019)",
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(9)),
        ticktext = ["0:00", "4:00", "9:00", "15:00", "17:00", "24:00"])
)

fig.show()

**Take-Aways:** From the *Plot*, it can be seen that the risk dips starts moderately high, dipping at 9am and moves up till 3pm then a slight dip at 5pm, later to only move up till midnight; However, our overall risk has been reduced drastically. This this option can practically reduce our risk, but it still follows the same risk patterns of the *current situation*. Further options must be explored to achive a risk of a flat $0$. 